In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
na_values=['NA', 'nan']

# Store labels as string because later we will concatinate them
dtype = {'fecha_dato':np.str, 'fecha_alta':np.str}

iter_csv = pd.read_csv('../data/raw/train.csv.zip', na_values = na_values, skipinitialspace = True, 
                       iterator=True, chunksize=100000, dtype=dtype)

corrupted = pd.concat([chunk for chunk in iter_csv])

/Users/petr/Applications/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/petr/Applications/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/petr/Applications/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/petr/Applications/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/petr/Applications/anaconda3/lib/python3

In [3]:
# Remove corrupted entries
print('%d users before cleaning' % len(corrupted.ncodpers.unique()))
corrupted = corrupted[corrupted.age.notnull()]
corrupted = corrupted[corrupted.sexo.notnull()]
print('%d users after cleaning' % len(corrupted.ncodpers.unique()))
corrupted.drop('nomprov', axis=1, inplace=True) # Drop province name. It is 100% useless

956645 users before cleaning
949609 users after cleaning


In [5]:
corrupted.sort_values(['ncodpers', 'fecha_dato'], inplace=True)
corrupted.drop_duplicates(['ncodpers'], keep='last', inplace=True)
corrupted.reset_index(drop=True, inplace=True)
X_train = corrupted

In [6]:
X_train.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2016-05-28,15889,F,ES,V,56.0,1995-01-16,0.0,255.0,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,1.0,326124.90,01 - TOP,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0
1,2016-05-28,15890,A,ES,V,63.0,1995-01-16,0.0,256.0,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,1.0,71461.20,01 - TOP,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1.0,1.0,1
2,2015-08-28,15891,N,ES,H,59.0,2015-07-28,0.0,246.0,99.0,2015-08-05,1,A,S,N,N,KAT,N,1.0,28.0,0.0,NaN,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2016-05-28,15892,F,ES,H,62.0,1995-01-16,0.0,256.0,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,1.0,430477.41,01 - TOP,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1
4,2016-05-28,15893,N,ES,V,63.0,1997-10-03,0.0,256.0,1.0,NaN,1,A,S,N,N,KAT,N,1.0,28.0,1.0,430477.41,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0


In [7]:
def sub_code_and_toint(text):
    return 0.0 if pd.isnull(text) else float(str(text)[1:2])
    
def preprocess_features(X):
    X.ind_empleado.replace(to_replace=[1.0, 99.0], value=[1.0, 0.0], inplace=True)
    
    X.indext.replace(to_replace=['S', 'N'], value=[1.0, 0.0], inplace=True)
    X.indresi.replace(to_replace=['S', 'N'], value=[1.0, 0.0], inplace=True)
    
    X.sexo.replace(to_replace=['V', 'H'], value=[1.0, 0.0], inplace=True)
    
    X.segmento = X.segmento.apply(sub_code_and_toint)

In [8]:
preprocess_features(X_train)
X_train.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2016-05-28,15889,F,ES,1.0,56.0,1995-01-16,0.0,255.0,1.0,NaN,1,A,1.0,0.0,N,KAT,N,1.0,28.0,1.0,326124.90,1.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0
1,2016-05-28,15890,A,ES,1.0,63.0,1995-01-16,0.0,256.0,1.0,NaN,1,A,1.0,0.0,N,KAT,N,1.0,28.0,1.0,71461.20,1.0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1.0,1.0,1
2,2015-08-28,15891,N,ES,0.0,59.0,2015-07-28,0.0,246.0,99.0,2015-08-05,1,A,1.0,0.0,N,KAT,N,1.0,28.0,0.0,NaN,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2016-05-28,15892,F,ES,0.0,62.0,1995-01-16,0.0,256.0,1.0,NaN,1,A,1.0,0.0,N,KAT,N,1.0,28.0,1.0,430477.41,1.0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1
4,2016-05-28,15893,N,ES,1.0,63.0,1997-10-03,0.0,256.0,1.0,NaN,1,A,1.0,0.0,N,KAT,N,1.0,28.0,1.0,430477.41,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0


In [10]:
test = pd.read_csv('../data/raw/test.csv.zip', usecols=['ncodpers'])

In [11]:
len(test[~test.ncodpers.isin(X_train.ncodpers)])

5

In [12]:
X_train.drop(['fecha_dato'], axis=1, inplace=True)
X_train.head()

,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,15889,F,ES,1.0,56.0,1995-01-16,0.0,255.0,1.0,NaN,1,A,1.0,0.0,N,KAT,N,1.0,28.0,1.0,326124.90,1.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0
1,15890,A,ES,1.0,63.0,1995-01-16,0.0,256.0,1.0,NaN,1,A,1.0,0.0,N,KAT,N,1.0,28.0,1.0,71461.20,1.0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1.0,1.0,1
2,15891,N,ES,0.0,59.0,2015-07-28,0.0,246.0,99.0,2015-08-05,1,A,1.0,0.0,N,KAT,N,1.0,28.0,0.0,NaN,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,15892,F,ES,0.0,62.0,1995-01-16,0.0,256.0,1.0,NaN,1,A,1.0,0.0,N,KAT,N,1.0,28.0,1.0,430477.41,1.0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1
4,15893,N,ES,1.0,63.0,1997-10-03,0.0,256.0,1.0,NaN,1,A,1.0,0.0,N,KAT,N,1.0,28.0,1.0,430477.41,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0


In [13]:
X_train.to_csv('../data/processed/train-processed.csv.gz', index=False, compression='gzip')

In [14]:
print(len(X_train.ncodpers))
print(len(X_train.ncodpers.unique()))

949609
949609
